In [17]:
# import libaries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression


In [18]:
#Importing data

data_path = r"C:\Users\masud\Documents\Immo_Eliza_House_App\Project\Data\data2.csv"
data= pd.read_csv (data_path, header= 1)
#data.head(5)                                               # At the point of data importation, the data has 10187 rows



In [19]:
data = data[data['Area'].str.contains('m²')]                                     # this check if the data contain "m2"
data['Area'] = data['Area'].str.split('|', n=1, expand=False)                    # this split the data by "|" into list (0, 1)


data['Area']=data.Area.apply(lambda x: x[0] if len(x) ==1 else x[1])            # This checked if the data len is 1, then X= x[0] else x[1]
data['Area'] = data['Area'].str.extract(r'(\d+)') 

data['Number_of_Rooms'] = data['Number_of_Rooms'].str.extract(r'(\d+)')         # using regrex (regular expressions)
data['Garden surface'] = data['Garden surface'].str.extract(r'(\d+)')

#data['Price'] = data['Price'].str.replace('€', '')

data                                                                   # At the end of this step, the data has drop from 10187 to 9222 rows

,Title,Location,Price,Area,Number_of_Rooms,Furnished,Kitchen type,Status,Garden surface,Swimming Pool
0,Villa for sale,Prins Karellaan 13,"€ 1,650",499,4,No,Hyper equipped,As new,450,No
1,House for sale,Lembergsesteenweg 67 Box lot 2,"€ 419,000",188,3,No,Hyper equipped,As new,330,No
2,House for sale,Lembergsesteenweg 67 Box lot 1,"€ 459,000",164,3,No,Hyper equipped,As new,341,No
3,House for sale,Ruelle Hierby 17,"€ 149,000",134,2,No,Installed,Good,300,No
4,House for sale,Rue Petite Voie 176,"€ 149,000",198,5,No,Semi equipped,To be done up,147,No
...,...,...,...,...,...,...,...,...,...,...
10182,Villa for sale,Laar 196,"€ 599,900",165,4,No,Installed,To be done up,751,No
10183,Mansion for sale,1030 — Schaerbeek,"€ 729,000",411,8,No,Installed,Good,751,No
10184,Apartment for sale,1370 — Jodoigne,"€ 150,000",68,1,No,Hyper equipped,To be done up,751,No
10185,Penthouse for sale,Kustlaan 111 Box 6/7,"€ 2,795",226,3,No,Hyper equipped,As new,751,Yes


In [20]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('€', '').replace(',', ''))
    return(x)

data['Price'] = data['Price'].apply(clean_currency).astype('float')
#data['Price_Type'] =data['Price'].apply(lambda x: type(x).__name__)

data

,Title,Location,Price,Area,Number_of_Rooms,Furnished,Kitchen type,Status,Garden surface,Swimming Pool
0,Villa for sale,Prins Karellaan 13,1650.0,499,4,No,Hyper equipped,As new,450,No
1,House for sale,Lembergsesteenweg 67 Box lot 2,419000.0,188,3,No,Hyper equipped,As new,330,No
2,House for sale,Lembergsesteenweg 67 Box lot 1,459000.0,164,3,No,Hyper equipped,As new,341,No
3,House for sale,Ruelle Hierby 17,149000.0,134,2,No,Installed,Good,300,No
4,House for sale,Rue Petite Voie 176,149000.0,198,5,No,Semi equipped,To be done up,147,No
...,...,...,...,...,...,...,...,...,...,...
10182,Villa for sale,Laar 196,599900.0,165,4,No,Installed,To be done up,751,No
10183,Mansion for sale,1030 — Schaerbeek,729000.0,411,8,No,Installed,Good,751,No
10184,Apartment for sale,1370 — Jodoigne,150000.0,68,1,No,Hyper equipped,To be done up,751,No
10185,Penthouse for sale,Kustlaan 111 Box 6/7,2795.0,226,3,No,Hyper equipped,As new,751,Yes


In [21]:
data['Furnished_encoded'] =data['Furnished'].map({'No': 0, 'Yes': 1})         
data['Swimming Pool_encoded'] =data['Swimming Pool'].map({'No': 0, 'Yes': 1})

data

,Title,Location,Price,Area,Number_of_Rooms,Furnished,Kitchen type,Status,Garden surface,Swimming Pool,Furnished_encoded,Swimming Pool_encoded
0,Villa for sale,Prins Karellaan 13,1650.0,499,4,No,Hyper equipped,As new,450,No,0,0
1,House for sale,Lembergsesteenweg 67 Box lot 2,419000.0,188,3,No,Hyper equipped,As new,330,No,0,0
2,House for sale,Lembergsesteenweg 67 Box lot 1,459000.0,164,3,No,Hyper equipped,As new,341,No,0,0
3,House for sale,Ruelle Hierby 17,149000.0,134,2,No,Installed,Good,300,No,0,0
4,House for sale,Rue Petite Voie 176,149000.0,198,5,No,Semi equipped,To be done up,147,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10182,Villa for sale,Laar 196,599900.0,165,4,No,Installed,To be done up,751,No,0,0
10183,Mansion for sale,1030 — Schaerbeek,729000.0,411,8,No,Installed,Good,751,No,0,0
10184,Apartment for sale,1370 — Jodoigne,150000.0,68,1,No,Hyper equipped,To be done up,751,No,0,0
10185,Penthouse for sale,Kustlaan 111 Box 6/7,2795.0,226,3,No,Hyper equipped,As new,751,Yes,0,1


In [22]:
#removing duplicate 

data.drop_duplicates()

,Title,Location,Price,Area,Number_of_Rooms,Furnished,Kitchen type,Status,Garden surface,Swimming Pool,Furnished_encoded,Swimming Pool_encoded
0,Villa for sale,Prins Karellaan 13,1650.0,499,4,No,Hyper equipped,As new,450,No,0,0
1,House for sale,Lembergsesteenweg 67 Box lot 2,419000.0,188,3,No,Hyper equipped,As new,330,No,0,0
2,House for sale,Lembergsesteenweg 67 Box lot 1,459000.0,164,3,No,Hyper equipped,As new,341,No,0,0
3,House for sale,Ruelle Hierby 17,149000.0,134,2,No,Installed,Good,300,No,0,0
4,House for sale,Rue Petite Voie 176,149000.0,198,5,No,Semi equipped,To be done up,147,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10176,Villa for sale,1860 — Meise,950000.0,318,5,No,Hyper equipped,Just renovated,2459,No,0,0
10178,House for sale,Pater Domstraat 16/W3,410000.0,161,4,No,Installed,As new,2459,No,0,0
10179,House for sale,Kastanjelaan 6,619000.0,304,6,No,Installed,As new,2459,No,0,0
10180,House for sale,Elsenstraat 12,215000.0,166,3,No,Installed,To be done up,2459,No,0,0


In [23]:
# Method 2 - Using the drop() function
data.drop(data.index[data['Price'] < 40000.0], inplace=True)


In [24]:
data.drop(data.index[data['Number_of_Rooms'] < "10"], inplace=True)

In [25]:
data

,Title,Location,Price,Area,Number_of_Rooms,Furnished,Kitchen type,Status,Garden surface,Swimming Pool,Furnished_encoded,Swimming Pool_encoded
1,House for sale,Lembergsesteenweg 67 Box lot 2,419000.0,188,3,No,Hyper equipped,As new,330,No,0,0
2,House for sale,Lembergsesteenweg 67 Box lot 1,459000.0,164,3,No,Hyper equipped,As new,341,No,0,0
3,House for sale,Ruelle Hierby 17,149000.0,134,2,No,Installed,Good,300,No,0,0
4,House for sale,Rue Petite Voie 176,149000.0,198,5,No,Semi equipped,To be done up,147,No,0,0
5,House for sale,Rue Pierre Curie 107,149000.0,80,2,No,Not installed,Good,147,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10180,House for sale,Elsenstraat 12,215000.0,166,3,No,Installed,To be done up,2459,No,0,0
10181,Villa for sale,Oostmalsesteenweg 295,635000.0,326,4,No,Installed,To be done up,900,No,0,0
10182,Villa for sale,Laar 196,599900.0,165,4,No,Installed,To be done up,751,No,0,0
10183,Mansion for sale,1030 — Schaerbeek,729000.0,411,8,No,Installed,Good,751,No,0,0


In [26]:
data.drop(['Furnished', 'Swimming Pool'], axis=1, inplace=True)

#Remane a column
dict = {'Swimming Pool_encoded': 'Swimming Pool',
        'Furnished_encoded': 'Furnished'}

# call rename () method
data.rename(columns=dict, inplace=True)     

data

,Title,Location,Price,Area,Number_of_Rooms,Kitchen type,Status,Garden surface,Furnished,Swimming Pool
1,House for sale,Lembergsesteenweg 67 Box lot 2,419000.0,188,3,Hyper equipped,As new,330,0,0
2,House for sale,Lembergsesteenweg 67 Box lot 1,459000.0,164,3,Hyper equipped,As new,341,0,0
3,House for sale,Ruelle Hierby 17,149000.0,134,2,Installed,Good,300,0,0
4,House for sale,Rue Petite Voie 176,149000.0,198,5,Semi equipped,To be done up,147,0,0
5,House for sale,Rue Pierre Curie 107,149000.0,80,2,Not installed,Good,147,0,0
...,...,...,...,...,...,...,...,...,...,...
10180,House for sale,Elsenstraat 12,215000.0,166,3,Installed,To be done up,2459,0,0
10181,Villa for sale,Oostmalsesteenweg 295,635000.0,326,4,Installed,To be done up,900,0,0
10182,Villa for sale,Laar 196,599900.0,165,4,Installed,To be done up,751,0,0
10183,Mansion for sale,1030 — Schaerbeek,729000.0,411,8,Installed,Good,751,0,0


In [27]:
# removing extra column that is not needed for the model 

data.drop(['Title', 'Location',"Status", "Kitchen type", "Furnished","Swimming Pool"], axis=1, inplace=True)


# Building model 


In [28]:
data

,Price,Area,Number_of_Rooms,Garden surface
1,419000.0,188,3,330
2,459000.0,164,3,341
3,149000.0,134,2,300
4,149000.0,198,5,147
5,149000.0,80,2,147
...,...,...,...,...
10180,215000.0,166,3,2459
10181,635000.0,326,4,900
10182,599900.0,165,4,751
10183,729000.0,411,8,751


In [29]:
#Setting the value for X and Y

x = data[['Area', 'Number_of_Rooms', 'Garden surface']]
y = data['Price']

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 100)

In [31]:
#Fitting the Multiple Linear Regression model
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state=0)
#model = LinearRegression()  
model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=0)

In [32]:
"""
#Intercept and Coefficient
print("Intercept: ", model.intercept_)
print("Coefficients:")
list(zip(x, model.coef_))
"""

'\n#Intercept and Coefficient\nprint("Intercept: ", model.intercept_)\nprint("Coefficients:")\nlist(zip(x, model.coef_))\n'

In [33]:
preidct= model.predict(X_test)

In [34]:
model.score(X_test, y_test)

0.4380334723601669

In [35]:
# Price = 215550.6426 + (753.0840 * Area) + (18.7893* Number_of_Rooms) + (0.02140 * Newspaper) + e   #Regression Equation